In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-TRAIN-80.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10_BALANCED.csv'

ONE_HOT_FORMAT = False

TRAINING_BATCH_SIZE = 32
TRAINING_DATA_AUG = True
TRAINING_BALANCED = False
TRAINING_PREFETCH = 20
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 32
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_PREFETCH = 1
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    one_hot_format=ONE_HOT_FORMAT,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE)

Se aplica data aug


## Class weights

In [8]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_true_train[:,0]), y=y_true_train[:,0])
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

{0: 0.6740610690645092, 1: 1.9362775165269543}


In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 268, 268
    tf.keras.layers.Conv2D(64,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 53, 53
    tf.keras.layers.Conv2D(32,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 26, 26
    tf.keras.layers.Conv2D(1,(3,3), strides=1),
    tf.keras.layers.ReLU(), # 24, 24
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation='relu'),
#     tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 268, 268, 32)      2432      
_________________________________________________________________
re_lu (ReLU)                 (None, 268, 268, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        100416    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 53, 53, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        18464     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 1)         2

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Create callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard('logs/MPropio_RedSeq_EYEPACS', histogram_freq=1, write_graph=False)

cbacks = [tensorboard_callback]

In [ ]:
# See Tensorboard
%tensorboard --logdir logs

In [11]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, clipnorm=1.0), 
              metrics=['accuracy']) 

num_epochs = 2000

history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=1, class_weight=d_class_weights)

Epoch 1/2000
1776/1776 [==============================] - 209s 118ms/step - loss: 0.6932 - accuracy: 0.4328 - val_loss: 0.6931 - val_accuracy: 0.5031
Epoch 2/2000
1776/1776 [==============================] - 201s 113ms/step - loss: 0.6926 - accuracy: 0.5294 - val_loss: 0.6911 - val_accuracy: 0.5409
Epoch 3/2000
1776/1776 [==============================] - 197s 111ms/step - loss: 0.6911 - accuracy: 0.5288 - val_loss: 0.6896 - val_accuracy: 0.5306
Epoch 4/2000
1776/1776 [==============================] - 199s 112ms/step - loss: 0.6896 - accuracy: 0.5320 - val_loss: 0.6862 - val_accuracy: 0.5503
Epoch 5/2000
1776/1776 [==============================] - 198s 112ms/step - loss: 0.6879 - accuracy: 0.5430 - val_loss: 0.6831 - val_accuracy: 0.5564
Epoch 6/2000
1776/1776 [==============================] - 198s 112ms/step - loss: 0.6863 - accuracy: 0.5470 - val_loss: 0.6835 - val_accuracy: 0.5559
Epoch 7/2000
1776/1776 [==============================] - 200s 112ms/step - loss: 0.6846 - accuracy:

KeyboardInterrupt: 